In [2]:
from pathlib import Path
path_folder = Path("/Users/keithlowton/Desktop/Ks/Python code/Kaggle/Fast Food Restaurants Across America")
print(path_folder.exists())
print(path_folder)

True
/Users/keithlowton/Desktop/Ks/Python code/Kaggle/Fast Food Restaurants Across America


In [27]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import folium
from pyproj import CRS
from shapely.geometry import Point
from geopy.distance import geodesic

### Functions

In [19]:
def create_point(df, Long, Lat):
    crs = CRS("epsg:4326")
    geometry = [Point(xy) for xy in zip(df[Long], df[Lat])]
    df = gpd.GeoDataFrame(df, geometry=geometry).set_crs(crs, allow_override=True)
    df.rename(columns={"geometry": "Centroid"}, inplace=True)
    return df

In [23]:
def create_buffer_area(df, geometry_col, radius_list):
    gdf = gpd.GeoDataFrame(df, geometry=geometry_col)
    gdf.to_crs(epsg=2272, inplace=True)
    rdf = gpd.GeoDataFrame()
    for r in radius_list:
        tadf = pd.merge(df, gpd.GeoDataFrame(gdf.geometry.buffer(r*5280)), left_index=True, right_index=True)
        tadf["Radius"], tadf["Radius_Type"] = r, "Mile"
        rdf = pd.concat([rdf, tadf])
    rdf.rename(columns={0: "Buffer_Area"}, inplace=True)
    rdf = gpd.GeoDataFrame(rdf, geometry=geometry_col)
    rdf.to_crs(epsg=4326, inplace=True)
    return rdf

In [33]:
def plot_point_polygon(df, latitude, longitude, trade_area):
    middle_long = np.median([df[longitude]])
    middle_lat = np.median([df[latitude]])
    m = folium.Map(location=[middle_lat, middle_long], width="80%", height="80%", tiles="OpenStreetMap", zoom_start=7)
    for idx, r in df.iterrows():
        folium.Marker([r[latitude], r[longitude]])
    polys = df[trade_area]
    folium.GeoJson(polys.to_crs(epsg=4326)).add_to(m)
    return m

In [37]:
def setgeometry(df, geometry_variable, crs_type):
    df["geometry"] = df[geometry_variable]
    df = df.set_geometry("geometry")
    df.to_crs(epsg=crs_type, inplace=True)
    return df

In [ ]:
def spatial_match(df1, geometry1_col, df2, geometry2_col, crs_type=2272, how_type="inner"):
    df1 = setgeometry(df1, geometry1_col, crs_type)
    df2 = setgeometry(df2, geometry2_col, crs_type)
    new_df = gpd.sjoin(df1, df2, how=how_type, op="intersects")
    return new_df

### Read data in

In [6]:
data = pd.read_csv(os.path.join(path_folder, "data/FastFoodRestaurants.csv"), sep=",")
print(data.shape)
    # (41188, 21)
data.head()

(10000, 10)


,address,city,country,keys,latitude,longitude,name,postalCode,province,websites
0,324 Main St,Massena,US,us/ny/massena/324mainst/-1161002137,44.92130,-74.89021,McDonald's,13662,NY,"http://mcdonalds.com,http://www.mcdonalds.com/..."
1,530 Clinton Ave,Washington Court House,US,us/oh/washingtoncourthouse/530clintonave/-7914...,39.53255,-83.44526,Wendy's,43160,OH,http://www.wendys.com
2,408 Market Square Dr,Maysville,US,us/ky/maysville/408marketsquaredr/1051460804,38.62736,-83.79141,Frisch's Big Boy,41056,KY,"http://www.frischs.com,https://www.frischs.com..."
3,6098 State Highway 37,Massena,US,us/ny/massena/6098statehighway37/-1161002137,44.95008,-74.84553,McDonald's,13662,NY,"http://mcdonalds.com,http://www.mcdonalds.com/..."
4,139 Columbus Rd,Athens,US,us/oh/athens/139columbusrd/990890980,39.35155,-82.09728,OMG! Rotisserie,45701,OH,"http://www.omgrotisserie.com,http://omgrotisse..."


In [7]:
df_data = pd.read_csv(os.path.join(path_folder, "data/Datafiniti_Fast_Food_Restaurants_May19.csv"), sep=",")
print(df_data.shape)
# (10000, 16)
df_data.head()

(10000, 16)


,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.39629,-104.69699,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.08135,-108.58689,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.09148,-108.55411,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.76369,-104.77671,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.74044,-104.92636,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...


In [8]:
df_data = pd.read_csv(os.path.join(path_folder, "data/Datafiniti_Fast_Food_Restaurants.csv"), sep=",")
print(df_data.shape)
# (10000, 16)
df_data.head()

(10000, 15)


,id,dateAdded,dateUpdated,address,categories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,American Restaurant and Fast Food Restaurant,Thibodaux,US,us/la/thibodaux/800ncanalblvd/1780593795,29.814697,-90.814742,SONIC Drive In,70301,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...
1,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,Fast Food Restaurants,Thibodaux,US,us/la/thibodaux/800ncanalblvd/1780593795,29.814697,-90.814742,SONIC Drive In,70301,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...
2,AVwcopQoByjofQCxgfVa,2016-03-29T05:06:36Z,2018-06-26T02:59:52Z,206 Wears Valley Rd,Fast Food Restaurant,Pigeon Forge,US,us/tn/pigeonforge/206wearsvalleyrd/-864103396,35.803788,-83.580553,Taco Bell,37863,TN,https://www.yellowpages.com/pigeon-forge-tn/mi...,"http://www.tacobell.com,https://locations.taco..."
3,AVweXN5RByjofQCxxilK,2017-01-03T07:46:11Z,2018-06-26T02:59:51Z,3652 Parkway,Fast Food,Pigeon Forge,US,us/tn/pigeonforge/3652parkway/93075755,35.782339,-83.551408,Arby's,37863,TN,http://www.yellowbook.com/profile/arbys_163389...,"http://www.arbys.com,https://locations.arbys.c..."
4,AWQ6MUvo3-Khe5l_j3SG,2018-06-26T02:59:43Z,2018-06-26T02:59:43Z,2118 Mt Zion Parkway,Fast Food Restaurant,Morrow,US,us/ga/morrow/2118mtzionparkway/1305117222,33.562738,-84.321143,Steak 'n Shake,30260,GA,https://foursquare.com/v/steak-n-shake/4bcf77a...,http://www.steaknshake.com/locations/23851-ste...


In [11]:
data = data[["name", "address", "city", "province", "latitude", "longitude"]]
data.rename(columns={"province": "state"}, inplace=True)
df_MD_NY = data[(data.state == "NY") & (data.name.isin(["McDonald\'s", "Mcdonald\'s", "McDonalds"]))]
df_MD_NY.reset_index(drop=True, inplace=True)

df_BK_NY = data[(data.state == "NY") & (data.name.isin(["Burger King"]))]
df_BK_NY.reset_index(drop=True, inplace=True)

In [20]:
df_MD_NY = create_point(df_MD_NY, "longitude", "latitude")
df_BK_NY = create_point(df_BK_NY, "longitude", "latitude")
df_MD_NY.head()

,name,address,city,state,latitude,longitude,Centroid
0,McDonald's,324 Main St,Massena,NY,44.921300,-74.89021,POINT (-74.89021 44.92130)
1,McDonald's,6098 State Highway 37,Massena,NY,44.950080,-74.84553,POINT (-74.84553 44.95008)
2,McDonald's,603 E Main St,Endicott,NY,42.095910,-76.05453,POINT (-76.05453 42.09591)
3,McDonald's,1134 Paterson St,Ogdensburg,NY,44.690205,-75.47740,POINT (-75.47740 44.69020)
4,McDonald's,51-67 E 161st St,Bronx,NY,40.828053,-73.92529,POINT (-73.92529 40.82805)


In [24]:
radii = [3]

df_MD_NY = create_buffer_area(df_MD_NY, "Centroid", radii)
df_MD_NY.head()

,name,address,city,state,latitude,longitude,Centroid,Buffer_Area,Radius,Radius_Type
0,McDonald's,324 Main St,Massena,NY,44.921300,-74.89021,POINT (-74.89021 44.92130),"POLYGON ((2727277.392 2050026.579, 2727201.118...",3,Mile
1,McDonald's,6098 State Highway 37,Massena,NY,44.950080,-74.84553,POINT (-74.84553 44.95008),"POLYGON ((2738534.378 2060925.091, 2738458.104...",3,Mile
2,McDonald's,603 E Main St,Endicott,NY,42.095910,-76.05453,POINT (-76.05453 42.09591),"POLYGON ((2444652.776 1010980.153, 2444576.502...",3,Mile
3,McDonald's,1134 Paterson St,Ogdensburg,NY,44.690205,-75.47740,POINT (-75.47740 44.69020),"POLYGON ((2576945.201 1961121.628, 2576868.927...",3,Mile
4,McDonald's,51-67 E 161st St,Bronx,NY,40.828053,-73.92529,POINT (-73.92529 40.82805),"POLYGON ((3042476.171 567435.222, 3042399.897 ...",3,Mile


In [34]:
ffr_map = plot_point_polygon(df_MD_NY, "latitude", "longitude", "Buffer_Area")
ffr_map

In [39]:
df_MD_NY.columns = ["MD_" + col for col in df_MD_NY.columns]
df_BK_NY.columns = ["BK_" + col for col in df_BK_NY.columns]

In [40]:
BK_near_MD_NY = spatial_match(df_MD_NY, "MD_Buffer_Area", df_BK_NY, "BK_Centroid")
BK_near_MD_NY = BK_near_MD_NY[["MD_latitude", "MD_longitude", "MD_Centroid", "MD_Buffer_Area", "BK_latitude", "BK_longitude", "BK_Centroid"]]
BK_near_MD_NY.reset_index(drop=True, inplace=True)
for idx, r in BK_near_MD_NY.iterrows():
    folium.CircleMarker([r["BK_latitude"], r["BK_longitude"]], radius=2, color="red").add_to(ffr_map)
ffr_map

/Users/keithlowton/anaconda3/envs/Geospatial/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
